# GMRES from scratch

In [47]:
import numpy as np

def gmres(A, b, x0, m=10, tol=1e-6, max_iter=100):

    n = A.shape[0]

    for iter_count in range(max_iter):
        # STEP 1: INITIALIZATION
        # Initial residual: r_0 = b - A x_0
        r0 = b - np.dot(A, x0)

        # Norm of initial residual: beta = ||r_0||_2
        beta = np.linalg.norm(r0)

        # Normalize the initial residual to get the first Krylov basis vector
        # q_1 = r_0 / beta
        q_vectors = np.zeros((n, m + 1))
        q_vectors[:, 0] = r0 / beta

        # Initialize the upper Hessenberg matrix H for the least-squares problem
        H = np.zeros((m + 1, m))

        # The right-hand side of the least-squares problem: beta * e_1
        # e_1 is the first standard basis vector in R^(m+1)
        e1 = np.zeros(m + 1)
        e1[0] = 1.0
        rhs = beta * e1

        # STEP 2: ARNOLDI ITERATION (BUILDING THE ORTHONORMAL BASIS)
        for j in range(m):
            # Generate the next vector: v = A q_j
            v = np.dot(A, q_vectors[:, j])

            # Gram-Schmidt Orthogonalization
            # Orthogonalize v against all previous q vectors (q_1 to q_j)
            for i in range(j + 1):
                # h_ij = q_i^T v
                H[i, j] = np.dot(q_vectors[:, i], v)
                # v = v - h_ij * q_i
                v = v - H[i, j] * q_vectors[:, i]

            # Normalize the new vector to get the next basis vector q_{j+1}
            # h_{j+1, j} = ||v||_2
            H[j + 1, j] = np.linalg.norm(v)

            # Avoid division by zero if the subspace is invariant
            if H[j + 1, j] < 1e-12: #the exact solution is found
                m = j + 1
                break

            # q_{j+1} = v / h_{j+1, j}
            q_vectors[:, j + 1] = v / H[j + 1, j]

        # STEP 3: SOLVE THE LEAST-SQUARES PROBLEM

        # The Arnoldi process gives us A @ Q_m = Q_{m+1} @ H_m
        # We need to find y that minimizes ||beta * e_1 - H_m @ y||_2

        # Truncate H
        H_m = H[:m + 1, :m]

        # Solve the small (m+1) x m least-squares problem
        y_m, _, _, _ = np.linalg.lstsq(H_m, rhs, rcond=None)

        # STEP 4: UPDATE THE SOLUTION
        # First, get the basis for K_m (the first m vectors of Q)
        # Q_m = q_vectors[:, :m]
        Q_m = q_vectors[:, :m]

        # Correction vector: deltax_m = Q_m @ y_m
        deltax_m = Q_m @ y_m

        # New solution: x_m = x_0 + z_m
        x0 = x0 + deltax_m

        # Check for convergence and prepare for next restart
        # The residual norm ||b - Ax_m|| is conveniently ||H_m @ y - beta * e_1||
        residual_norm = np.linalg.norm(H_m @ y_m - rhs)
        print(f"Iteration {iter_count+1}, Restart Cycle {m}, Residual Norm: {residual_norm:.6e}")

        if residual_norm < tol:
            print(f"Converged after {iter_count+1} outer iterations.")
            return x0, residual_norm

    print("GMRES failed to converge within the maximum number of iterations.")
    return x0, residual_norm

In [48]:
N = 100
np.random.seed(0)
A_rand = np.random.rand(N, N)
A = A_rand + np.diag(np.full(N, 10.0))

# Known solution
x_true = np.arange(N)

b = np.dot(A, x_true)

x0_guess = np.zeros(N)
tolerance = 1e-8

m_restart = 20

solution, final_residual = gmres(A, b, x0_guess, m=m_restart, tol=tolerance)

error = np.linalg.norm(solution - x_true) / np.linalg.norm(x_true)
print()

print(f"Final Residual Norm: {final_residual:.6e}")
print(f"Relative Error in Solution: {error:.6e}")

Iteration 1, Restart Cycle 20, Residual Norm: 8.233172e-08
Iteration 2, Restart Cycle 20, Residual Norm: 7.014367e-19
Converged after 2 outer iterations.

Final Residual Norm: 7.014367e-19
Relative Error in Solution: 5.668363e-16


# GMRES in SciPy

In [44]:
from scipy.sparse.linalg import gmres as sp_gmres

scipy_solution, _ = sp_gmres(A, b, x0=x0_guess, atol=tolerance, restart=m_restart)

In [46]:
scipy_solution

array([1.66009705e-03, 9.98771389e-01, 2.00007155e+00, 3.00096251e+00,
       3.99941380e+00, 5.00050600e+00, 6.00073363e+00, 6.99868246e+00,
       7.99969566e+00, 8.99819499e+00, 1.00009991e+01, 1.09991457e+01,
       1.19993304e+01, 1.30014569e+01, 1.40011207e+01, 1.50026509e+01,
       1.60000016e+01, 1.70021734e+01, 1.79992596e+01, 1.90000674e+01,
       2.00010547e+01, 2.09982784e+01, 2.20013979e+01, 2.29987073e+01,
       2.40007510e+01, 2.50018323e+01, 2.60006197e+01, 2.70016449e+01,
       2.79998293e+01, 2.90004925e+01, 2.99987516e+01, 3.10007375e+01,
       3.19990582e+01, 3.29994651e+01, 3.39991342e+01, 3.49985255e+01,
       3.60011310e+01, 3.69998776e+01, 3.80000417e+01, 3.90012118e+01,
       3.99990560e+01, 4.10002645e+01, 4.19980618e+01, 4.29984735e+01,
       4.39967427e+01, 4.49983189e+01, 4.59976786e+01, 4.70008110e+01,
       4.80017881e+01, 4.89997823e+01, 4.99987770e+01, 5.09997332e+01,
       5.19989833e+01, 5.30009842e+01, 5.40002332e+01, 5.50016333e+01,
      